## Trying to run eval

```
"a0": "Because Sheldon is being rude.", 
"a1": "Because he doesn't like Sheldon.", 
"a2": "Because they are having an argument.", 
"a3": "Because Howard wanted to have a private meal with Raj.", 
"a4": "Because Sheldon won't loan him money for food.", 
"answer_idx": 2, 
"q": "Why is Howard frustrated when he is talking to Sheldon?", 
"qid": 122039, 
"show_name": "The Big Bang Theory", 
"ts": "20.16-25.12", 
"vid_name": "s03e02_seg02_clip_10"


"Question: {q} Is it '{ans_candidate}'? [MASK]."


Iterate thru, run embeddings. Then run construct question, then run model, find the highest probability YES answer. Return that answer_idx. Find accuacy.
```

In [7]:
import json
from typing import Dict

In [10]:
# load json from file
train_filepath="/mnt/teton/vpt/data/benchmark_datasets/TVQA/TVQA/data/tvqa_qa_release/tvqa_train.jsonl"
with open(train_filepath, 'r') as f:
    train_qa_json = [json.loads(line) for line in f]
len(train_qa_json)  # 122,039


122039

In [11]:
train_qa_json[0]

{'a0': 'Cafeteria',
 'a1': 'Hallway',
 'a2': 'Car',
 'a3': 'Patients room',
 'a4': 'Outside',
 'answer_idx': 4,
 'q': 'Where is Meredith when George approaches her?',
 'qid': 0,
 'show_name': "Grey's Anatomy",
 'ts': '76.01-84.2',
 'vid_name': 'grey_s03e20_seg02_clip_14'}

In [34]:
train_qa_json[0]['ts'].split('-')[0]

'76.01'

In [15]:

def qa_to_prompt(qa:Dict, captions:str):
    # prompts = []
    # for ans_candidate in [qa['a0'], qa['a1'], qa['a2'], qa['a3'], qa['a4']]:
    #     prompts.append([f"Question: {qa['q']} Is it '{ans_candidate}'?"])
    return [[f"Some context: {captions}. Question: {qa['q']} Is it '{ans_candidate}'?"] for ans_candidate in [qa['a0'], qa['a1'], qa['a2'], qa['a3'], qa['a4']]]


In [16]:
qa_to_prompt(train_qa_json[0], "some captions")

[["Some context: some captions. Question: Where is Meredith when George approaches her? Is it 'Cafeteria'?"],
 ["Some context: some captions. Question: Where is Meredith when George approaches her? Is it 'Hallway'?"],
 ["Some context: some captions. Question: Where is Meredith when George approaches her? Is it 'Car'?"],
 ["Some context: some captions. Question: Where is Meredith when George approaches her? Is it 'Patients room'?"],
 ["Some context: some captions. Question: Where is Meredith when George approaches her? Is it 'Outside'?"]]

In [ ]:
# todo: include captions in the prompt.

## Subtitles

In [17]:
# load json from file
subtitles_filepath="/mnt/teton/vpt/data/benchmark_datasets/TVQA/TVQA/data/tvqa_preprocessed_subtitles.jsonl"
with open(subtitles_filepath, 'r') as f:
    subtitles = [json.loads(line) for line in f]
len(subtitles)  # 21,793

21793

In [33]:
# subtitles[0]

def get_subtitle_from_clip(vid_name:str, start_time:float, end_time:float):
  subtitle = ''
  for sub in subtitles:
      if sub['vid_name'] == vid_name:
          # print()
          for text in sub['sub']:
              if text['start'] >= float(start_time - 1) and text['end'] <= float(end_time + 1):
                subtitle += text['text'] + ' '
                # print(text['text'])
                # print(text['start'], text['end'])
                # print()
  return subtitle.strip()

get_subtitle_from_clip('grey_s03e20_seg02_clip_14', 76.01, 84.2)


"Meredith! Tell me where she is. - I can't! If she wanted you to know... - She does. She..."

In [50]:
# testing
v = [
    "Some context: Meredith! Tell me where she is. - I can't! If she wanted you to know... - She does. She.... Question: Where is Meredith when George approaches her? Is it 'Cafeteria'?",
    "Some context: Meredith! Tell me where she is. - I can't! If she wanted you to know... - She does. She.... Question: Where is Meredith when George approaches her? Is it 'Hallway'?",
    "Some context: Meredith! Tell me where she is. - I can't! If she wanted you to know... - She does. She.... Question: Where is Meredith when George approaches her? Is it 'Car'?",
    "Some context: Meredith! Tell me where she is. - I can't! If she wanted you to know... - She does. She.... Question: Where is Meredith when George approaches her? Is it 'Patients room'?",
    "Some context: Meredith! Tell me where she is. - I can't! If she wanted you to know... - She does. She.... Question: Where is Meredith when George approaches her? Is it 'Outside'?"
]
v

["Some context: Meredith! Tell me where she is. - I can't! If she wanted you to know... - She does. She.... Question: Where is Meredith when George approaches her? Is it 'Cafeteria'?",
 "Some context: Meredith! Tell me where she is. - I can't! If she wanted you to know... - She does. She.... Question: Where is Meredith when George approaches her? Is it 'Hallway'?",
 "Some context: Meredith! Tell me where she is. - I can't! If she wanted you to know... - She does. She.... Question: Where is Meredith when George approaches her? Is it 'Car'?",
 "Some context: Meredith! Tell me where she is. - I can't! If she wanted you to know... - She does. She.... Question: Where is Meredith when George approaches her? Is it 'Patients room'?",
 "Some context: Meredith! Tell me where she is. - I can't! If she wanted you to know... - She does. She.... Question: Where is Meredith when George approaches her? Is it 'Outside'?"]

In [48]:
print("\n".join(v[:]))

TypeError: sequence item 0: expected str instance, list found

In [ ]:
# so now we have the text. Need to embed it. Then embed images. then we're ready to rock.

## Get logits for the generated answer

In [45]:
from transformers import AutoTokenizer, T5ForConditionalGeneration

model_name = 'google/flan-t5-small'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

In [88]:
prompt = "Some context: Meredith! Tell me where she is. - I can't! If she wanted you to know... - She does. She.... Question: Where is Meredith when George approaches her? Is it 'Cafeteria'?"
import numpy as np
all_yes_scores = []
for prompt in v:
  print(prompt)
  inputs = tokenizer(prompt, return_tensors="pt")  # Batch size 1
  outputs = model.generate(**inputs, return_dict_in_generate=True, output_scores=True, early_stopping=True, max_new_tokens=2)
  # print(outputs)
  all_yes_scores.append(outputs.scores[0][0][4273])
  print('yes_score =', outputs.scores[0][0][4273])
  print('no_score =', outputs.scores[0][0][150])
  print(tokenizer.batch_decode(outputs.sequences, skip_special_tokens=False))
  
print("Arg of 'yes' answer")
print(np.array(all_yes_scores).argmax())

Some context: Meredith! Tell me where she is. - I can't! If she wanted you to know... - She does. She.... Question: Where is Meredith when George approaches her? Is it 'Cafeteria'?
yes_score = tensor(-1.9372)
no_score = tensor(1.0013)
['<pad> no</s>']
Some context: Meredith! Tell me where she is. - I can't! If she wanted you to know... - She does. She.... Question: Where is Meredith when George approaches her? Is it 'Hallway'?
yes_score = tensor(-1.9553)
no_score = tensor(0.8167)
['<pad> no</s>']
Some context: Meredith! Tell me where she is. - I can't! If she wanted you to know... - She does. She.... Question: Where is Meredith when George approaches her? Is it 'Car'?
yes_score = tensor(-2.8168)
no_score = tensor(0.5007)
['<pad> no</s>']
Some context: Meredith! Tell me where she is. - I can't! If she wanted you to know... - She does. She.... Question: Where is Meredith when George approaches her? Is it 'Patients room'?
yes_score = tensor(-1.0733)
no_score = tensor(1.5197)
['<pad> no</s

In [74]:
# pytorch set max print size to 1000

outputs.scores[0][:5]

tensor([[-52.7880,  -5.7398, -11.3355,  ..., -52.8435, -52.7438, -52.6754]])

In [70]:
import torch
ord = torch.argsort(outputs.scores[0], descending=True)
ord

tensor([[  150,  1067,    27,  ..., 32065, 32114, 21515]])

In [67]:
outputs

GreedySearchEncoderDecoderOutput(sequences=tensor([[  0, 150,   1]]), scores=(tensor([[-52.7880,  -5.7398, -11.3355,  ..., -52.8435, -52.7438, -52.6754]]), tensor([[-71.3249,  -0.3261, -18.2158,  ..., -71.4193, -71.3626, -71.4082]])), encoder_attentions=None, encoder_hidden_states=None, decoder_attentions=None, cross_attentions=None, decoder_hidden_states=None)

In [73]:
print(tokenizer.batch_decode(ord[0][:5], skip_special_tokens=False))

['no', 'outside', 'I', '', 'yes']


In [75]:
ord[0][:5]

tensor([ 150, 1067,   27,    3, 4273])

In [84]:
# no:  150
# yes: 4273

print(outputs.scores[0][0][4273]) # yes
# print(outputs.scores[0][0][1067])  # outside
print(outputs.scores[0][0][150])  # no

# closer to 0 is better (smaller absolute value)

tensor(-2.7644)
tensor(-2.0362)
tensor(-1.2458)
